# Binary compound formation energy prediction example

This notebook demonstrates how to create a probabilistic model for predicting
formation energies of binary compounds with a quantified uncertainty. Before
running this notebook, ensure that you have a valid Materials Project API key
from <https://www.materialsproject.org/dashboard>. Next, either put this
key in a `.config` file, or change `MAPI_KEY` to the key.

<div class="alert alert-block alert-warning">
Be careful not to include API keys in published versions of this notebook!
</div>


In [1]:
import shutil
from pathlib import Path

import pandas as pd
from megnet.models import MEGNetModel
from pymatgen.ext.matproj import MPRester
from tensorflow.keras.callbacks import TensorBoard
from unlockgnn import MEGNetProbModel
from unlockgnn.metrics import evaluate_uq_metrics


In [2]:
THIS_DIR = Path(".").parent
CONFIG_FILE = THIS_DIR / ".config"

MAPI_KEY = None
MODEL_SAVE_DIR: Path = THIS_DIR / "binary_e_form_model"
DATA_SAVE_DIR: Path = THIS_DIR / "binary_data.pkl"
LOG_DIR = THIS_DIR / "logs"
BATCH_SIZE: int = 128
NUM_INDUCING_POINTS: int = 500
OVERWRITE: bool = True
TRAINING_RATIO: float = 0.8

if OVERWRITE:
    for directory in [MODEL_SAVE_DIR, LOG_DIR]:
        if directory.exists():
            shutil.rmtree(directory)

try:
    mp_key = CONFIG_FILE.read_text()
except FileNotFoundError:
    if MAPI_KEY is None:
        raise ValueError("Enter Materials Project API key either in a `.config` file or in the notebook itself.")
    mp_key = MAPI_KEY


# Data gathering

Here we download binary compounds that lie on the convex hull from the Materials
Project, then split them into training and validation subsets.


In [3]:
query = {
    "criteria": {"nelements": 2, "e_above_hull": 0},
    "properties": ["structure", "formation_energy_per_atom"],
}

if DATA_SAVE_DIR.exists():
    full_df = pd.read_pickle(DATA_SAVE_DIR)
else:
    with MPRester(mp_key) as mpr:
        full_df = pd.DataFrame(mpr.query(**query))
    full_df.to_pickle(DATA_SAVE_DIR)


In [4]:
full_df.head()

,structure,formation_energy_per_atom
0,"[[ 1.982598 -4.08421341 3.2051745 ] La, [1....",-0.737439
1,"[[0. 0. 0.] Fe, [1.880473 1.880473 1.880473] H]",-0.068482
2,"[[1.572998 0. 0. ] Ta, [0. ...",-0.773151
3,"[[0. 0. 7.42288687] Hf, [0. ...",-0.177707
4,"[[ 1.823716 -3.94193291 3.47897025] Tm, [1....",-0.905038


In [5]:
num_training = int(TRAINING_RATIO * len(full_df.index))
train_df = full_df[:num_training]
val_df = full_df[num_training:]

print(f"{num_training} training samples, {len(val_df.index)} validation samples.")

train_structs = train_df["structure"]
val_structs = val_df["structure"]

train_targets = train_df["formation_energy_per_atom"]
val_targets = val_df["formation_energy_per_atom"]


4217 training samples, 1055 validation samples.


# Model creation

Now we load the `MEGNet` 2019 formation energies model, then convert this to a
probabilistic model.


In [6]:
meg_model = MEGNetModel.from_mvl_models("Eform_MP_2019")


INFO:megnet.utils.models:Package-level mvl_models not included, trying temperary mvl_models downloads..
INFO:megnet.utils.models:Model found in local mvl_models path


Instructions for updating:
The `validate_indices` argument has no effect. Indices are always validated on CPU and never validated on GPU.


Instructions for updating:
The `validate_indices` argument has no effect. Indices are always validated on CPU and never validated on GPU.


In [7]:
kl_weight = BATCH_SIZE / num_training

prob_model = MEGNetProbModel(
    num_inducing_points=NUM_INDUCING_POINTS,
    save_path=MODEL_SAVE_DIR,
    meg_model=meg_model,
    kl_weight=kl_weight,
)


INFO:tensorflow:Assets written to: binary_e_form_model/megnet/assets


/home/awsm/anaconda3/envs/unlockGNN/lib/python3.8/site-packages/tensorflow/python/keras/utils/generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '
INFO:tensorflow:Assets written to: binary_e_form_model/megnet/assets


INFO:tensorflow:Assets written to: binary_e_form_model/gnn/assets


/home/awsm/anaconda3/envs/unlockGNN/lib/python3.8/site-packages/tensorflow/python/keras/utils/generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '
INFO:tensorflow:Assets written to: binary_e_form_model/gnn/assets


Instructions for updating:
`jitter` is deprecated; please use `marginal_fn` directly.


Instructions for updating:
`jitter` is deprecated; please use `marginal_fn` directly.
/home/awsm/anaconda3/envs/unlockGNN/lib/python3.8/site-packages/tensorflow_probability/python/distributions/gaussian_process.py:363: UserWarning: Unable to detect statically whether the number of index_points is 1. As a result, defaulting to treating the marginal GP at `index_points` as a multivariate Gaussian. This makes some methods, like `cdf` unavailable.
  warnings.warn(


# Train the uncertainty quantifier

Now we train the model. By default, the `MEGNet` (GNN) layers of the model are
frozen after initialization. Therefore, when we call `prob_model.train()`, the
only layers that are optimized are the `VariationalGaussianProcess` (VGP) and the
`BatchNormalization` layer (`Norm`) that feeds into it.

After this initial training, we unfreeze _all_ the layers and train the full model simulateously.


In [8]:
tb_callback_1 = TensorBoard(log_dir=LOG_DIR / "vgp_training", write_graph=False)
tb_callback_2 = TensorBoard(log_dir=LOG_DIR / "fine_tuning", write_graph=False)


In [9]:
%load_ext tensorboard
%tensorboard --logdir logs

In [10]:
prob_model.train(
    train_structs,
    train_targets,
    epochs=50,
    val_structs=val_structs,
    val_targets=val_targets,
    callbacks=[tb_callback_1],
)
prob_model.save()


Epoch 1/50


/home/awsm/anaconda3/envs/unlockGNN/lib/python3.8/site-packages/tensorflow_probability/python/distributions/gaussian_process.py:363: UserWarning: Unable to detect statically whether the number of index_points is 1. As a result, defaulting to treating the marginal GP at `index_points` as a multivariate Gaussian. This makes some methods, like `cdf` unavailable.
  warnings.warn(
/home/awsm/anaconda3/envs/unlockGNN/lib/python3.8/site-packages/tensorflow/python/framework/indexed_slices.py:447: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/model/set2_set_1/Reshape_9:0", shape=(None,), dtype=int32), values=Tensor("gradient_tape/model/set2_set_1/Reshape_8:0", shape=(None,), dtype=float32), dense_shape=Tensor("gradient_tape/model/set2_set_1/Cast:0", shape=(1,), dtype=int32))) to a dense Tensor of unknown shape. This may consume a large amount of memory.
  warnings.warn(
/home/awsm/anaconda3/envs/unlockGNN/lib/python3.8/site-packages/tensorflow/python/f

33/33 - 19s - loss: 2040835.1250 - val_loss: 1886394.0000
Epoch 2/50
33/33 - 9s - loss: 1890396.6250 - val_loss: 1722113.5000
Epoch 3/50
33/33 - 9s - loss: 1607083.1250 - val_loss: 1349981.2500
Epoch 4/50
33/33 - 9s - loss: 1222383.6250 - val_loss: 955772.6875
Epoch 5/50
33/33 - 9s - loss: 878158.5000 - val_loss: 623919.1250
Epoch 6/50
33/33 - 9s - loss: 590655.4375 - val_loss: 360427.2500
Epoch 7/50
33/33 - 9s - loss: 364731.6250 - val_loss: 184351.3125
Epoch 8/50
33/33 - 8s - loss: 221803.8438 - val_loss: 113347.5859
Epoch 9/50
33/33 - 9s - loss: 159917.9688 - val_loss: 83589.1562
Epoch 10/50
33/33 - 9s - loss: 127261.2578 - val_loss: 69015.7344
Epoch 11/50
33/33 - 9s - loss: 95602.8047 - val_loss: 58286.9648
Epoch 12/50
33/33 - 9s - loss: 88059.0234 - val_loss: 51375.9141
Epoch 13/50
33/33 - 9s - loss: 73762.1484 - val_loss: 46060.8203
Epoch 14/50
33/33 - 9s - loss: 66971.9453 - val_loss: 43101.3164
Epoch 15/50
33/33 - 9s - loss: 56670.4688 - val_loss: 38042.2930
Epoch 16/50
33/33 -

In [11]:
prob_model.set_frozen(["GNN", "VGP"], freeze=False)


In [12]:
prob_model.train(
    train_structs,
    train_targets,
    epochs=50,
    val_structs=val_structs,
    val_targets=val_targets,
    callbacks=[tb_callback_2],
)
prob_model.save()


Epoch 1/50
33/33 - 21s - loss: 57860.1680 - val_loss: 323303.4062
Epoch 2/50
33/33 - 9s - loss: 33428.8008 - val_loss: 45801.1406
Epoch 3/50
33/33 - 9s - loss: 23308.4512 - val_loss: 26309.8672
Epoch 4/50
33/33 - 9s - loss: 21981.1074 - val_loss: 29651.7012
Epoch 5/50
33/33 - 9s - loss: 17631.1172 - val_loss: 16150.3232
Epoch 6/50
33/33 - 9s - loss: 17124.3574 - val_loss: 11733.7979
Epoch 7/50
33/33 - 9s - loss: 19093.7812 - val_loss: 19135.8906
Epoch 8/50
33/33 - 9s - loss: 14067.5352 - val_loss: 11712.0547
Epoch 9/50
33/33 - 9s - loss: 10313.1748 - val_loss: 13322.2100
Epoch 10/50
33/33 - 9s - loss: 10201.9355 - val_loss: 9943.5732
Epoch 11/50
33/33 - 9s - loss: 10354.8184 - val_loss: 8510.2705
Epoch 12/50
33/33 - 9s - loss: 12133.2998 - val_loss: 9315.1436
Epoch 13/50
33/33 - 9s - loss: 10313.0898 - val_loss: 10559.1055
Epoch 14/50
33/33 - 9s - loss: 9158.5723 - val_loss: 9083.2881
Epoch 15/50
33/33 - 9s - loss: 9402.1826 - val_loss: 9860.4131
Epoch 16/50
33/33 - 9s - loss: 10662.23

# Model evaluation

Finally, we'll evaluate model metrics and make some sample predictions! Note that the predictions give predicted values and standard deviations. The standard deviations can then be converted to an uncertainty;
in this example, we'll take the uncertainty as twice the standard deviation, which will give us the 95% confidence interval (see <https://en.wikipedia.org/wiki/68%E2%80%9395%E2%80%9399.7_rule>).


In [13]:
example_structs = val_structs[:10].tolist()
example_targets = val_targets[:10].tolist()

predicted, stddevs = prob_model.predict(example_structs)
uncerts = 2 * stddevs


In [14]:
pd.DataFrame(
    {
        "Composition": [struct.composition.reduced_formula for struct in example_structs],
        "Formation energy per atom / eV": example_targets,
        "Predicted / eV": [
            f"{pred:.2f} ± {uncert:.2f}" for pred, uncert in zip(predicted, uncerts)
        ],
    }
)


,Composition,Formation energy per atom / eV,Predicted / eV
0,Zr2Cu,-0.132384,-0.09 ± 0.03
1,NbRh,-0.401313,-0.44 ± 0.05
2,Cu3Ge,-0.005707,-0.03 ± 0.04
3,Pr3In,-0.273232,-0.29 ± 0.03
4,InS,-0.742895,-0.78 ± 0.05
5,TmPb3,-0.215892,-0.25 ± 0.03
6,InNi,-0.174754,-0.13 ± 0.03
7,GdGe,-0.857117,-0.86 ± 0.07
8,GdTl,-0.380423,-0.38 ± 0.03
9,HoTl3,-0.215986,-0.21 ± 0.03


In [15]:
val_metrics = evaluate_uq_metrics(prob_model, val_structs, val_targets)
train_metrics = evaluate_uq_metrics(prob_model, train_structs, train_targets)

print(f"{val_metrics=}")
print(f"{train_metrics=}")


/home/awsm/anaconda3/envs/unlockGNN/lib/python3.8/site-packages/tensorflow_probability/python/distributions/gaussian_process.py:363: UserWarning: Unable to detect statically whether the number of index_points is 1. As a result, defaulting to treating the marginal GP at `index_points` as a multivariate Gaussian. This makes some methods, like `cdf` unavailable.
  warnings.warn(


val_metrics={'nll': 1274.4541579063027, 'sharpness': 0.031452605507070706, 'variation': 0.5576342435625041, 'mae': 0.048892347989350424, 'mse': 0.005417255757233144, 'rmse': 0.07360200919290957}
train_metrics={'nll': -7537.376558210598, 'sharpness': 0.03125057564791146, 'variation': 0.5446802327284079, 'mae': 0.02896238446924681, 'mse': 0.00164909309210959, 'rmse': 0.040609027224369584}
